In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!unzip -q /content/drive/My\ Drive/lower_zip.zip

In [0]:
!pip install split-folders

In [0]:
import split_folders
split_folders.ratio("/content/lower",output="/content/lower_data",ratio=(.7,.1,.2))

Copying files: 45190 files [00:06, 6690.85 files/s]


In [0]:
import torch
torch.cuda.current_device()

0

In [0]:
import torch.nn as nn
from torchvision import datasets
from torchvision.transforms import transforms
import torchvision.models as models
import numpy as np
from tqdm import tqdm
from PIL import Image
from PIL import ImageFile
use_cuda = torch.cuda.is_available()

In [0]:
img_std = [0.229,0.224,0.225]
img_mean = [0.485,0.456,0.406]
batch_size = 64

train_transforms = transforms.Compose([
                                transforms.ToTensor(),
                                transforms.Normalize(img_mean,img_std)
                                     ])
train_dir = "/content/lower_data/train"
valid_dir = "/content/lower_data/val"
test_dir = "/content/lower_data/test"

train_dataset = datasets.ImageFolder(train_dir,transform=train_transforms)
valid_dataset = datasets.ImageFolder(valid_dir,transform=train_transforms)
test_dataset = datasets.ImageFolder(test_dir,transform=train_transforms)

trainloader = torch.utils.data.DataLoader(train_dataset,batch_size=batch_size,shuffle=True)
validloader = torch.utils.data.DataLoader(valid_dataset,batch_size=batch_size,shuffle=True)
testloader = torch.utils.data.DataLoader(test_dataset,batch_size=batch_size,shuffle=True)

loaders = {
    'train':trainloader,
    'valid':validloader,
    'test':testloader
}

In [0]:
class LowerModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.encoder_net = models.resnet101(pretrained=True)
    self.encoder_net = nn.Sequential(*list(self.encoder_net.children())[:-1])
    
    self.fc1 = nn.Sequential(nn.Dropout(0.4),
                            nn.Linear(2048,256),
                            nn.BatchNorm1d(num_features=256),
                            nn.ReLU(True),
                            nn.Dropout(0.1),
                            nn.Linear(256,8),
                            nn.LogSoftmax(dim=1))
    
  def forward(self,x):
    encoding = self.encoder_net(x).view((-1,2048))
    preds = self.fc1(encoding)

    return preds
    
model = LowerModel()
if use_cuda:
  model.cuda()
model.load_state_dict(torch.load("/content/drive/My Drive/lower_3.pt"))

Downloading: "https://download.pytorch.org/models/resnet101-5d3b4d8f.pth" to /root/.cache/torch/checkpoints/resnet101-5d3b4d8f.pth
100%|██████████| 178728960/178728960 [00:06<00:00, 29079974.58it/s]


In [0]:
import torch.optim as optim

criterion = nn.NLLLoss()

optimizer = optim.Adam(model.parameters(),lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer,step_size=3,gamma=0.2)

In [0]:
def train(n_epochs,model,criterion,optimizer,scheduler,loaders,use_cuda,save_path):
  valid_loss_min = np.Inf
  valid_acc_min = 0
  
  for epoch in range(1,n_epochs+1):
    valid_loss = 0
    train_loss = 0
    valid_corrects = 0
    train_corrects = 0
    
    scheduler.step()
    
    # Training loop
    model.train()
    for data,target in tqdm(loaders['train']):
      if use_cuda:
        data,target = data.cuda(),target.cuda()
        
      optimizer.zero_grad()
      
      output = model(data)
      
      loss = criterion(output,target)
      
      loss.backward()
      
      optimizer.step()
      
      train_loss += loss.item()
      
      pred = output.data.max(1,keepdim=True)[1]
      train_corrects += np.sum(np.squeeze(pred.eq(target.data.view_as(pred))).cpu().numpy())
      
    
    #Validation loop
    model.eval()
    for data,target in tqdm(loaders['valid']):
      if use_cuda:
        data,target = data.cuda(),target.cuda()
       
      with torch.no_grad():
        output = model(data)
      
      loss = criterion(output,target)
      
      valid_loss += loss.item()
      
      pred = output.data.max(1,keepdim=True)[1]
      valid_corrects += np.sum(np.squeeze(pred.eq(target.data.view_as(pred))).cpu().numpy())
    
    valid_loss = valid_loss / len(loaders['valid'])
    train_loss = train_loss / len(loaders['train'])
    valid_accuracy = valid_corrects / (len(loaders['valid'])*batch_size)
    train_accuracy = train_corrects / (len(loaders['train'])*batch_size)
      
    #Print epoch details
    print("Epoch: {}, Training loss: {}, Training accuracy: {}, Valid loss: {}, Valid accuracy; {}".format(epoch,train_loss,train_accuracy,valid_loss,valid_accuracy))
    
    if valid_loss < valid_loss_min:
      valid_loss_min = valid_loss
      valid_acc_min = valid_accuracy
      checkpoint = model.state_dict()
      torch.save(checkpoint,save_path)
      
  print("The best model had a validation loss of {} and an accuracy of {}".format(valid_loss_min,valid_acc_min))

In [0]:
model = train(15,model,criterion,optimizer,scheduler,loaders,use_cuda,"/content/drive/My Drive/lower_3.pt")

100%|██████████| 71/71 [00:22<00:00,  3.68it/s]


Epoch: 1, Training loss: 0.6312787810961406, Training accuracy: 0.803125, Valid loss: 0.5889344903784739, Valid accuracy; 0.8076584507042254


100%|██████████| 71/71 [00:22<00:00,  3.66it/s]


Epoch: 2, Training loss: 0.4497705571278177, Training accuracy: 0.8613320707070707, Valid loss: 0.43182792567031486, Valid accuracy; 0.8657570422535211


100%|██████████| 71/71 [00:21<00:00,  3.71it/s]


...

Epoch: 13, Training loss: 0.04349165569351177, Training accuracy: 0.9882575757575758, Valid loss: 0.26192085565605633, Valid accuracy; 0.9262764084507042


  0%|          | 0/495 [00:00<?, ?it/s]

Epoch: 14, Training loss: 0.041965647564843445, Training accuracy: 0.9880997474747475, Valid loss: 0.2647376997579991, Valid accuracy; 0.926056338028169


100%|██████████| 71/71 [00:22<00:00,  3.68it/s]

Epoch: 15, Training loss: 0.04185700525689607, Training accuracy: 0.9888573232323232, Valid loss: 0.26480908392810487, Valid accuracy; 0.9258362676056338
The best model had a validation loss of 0.2499830701821287 and an accuracy of 0.9271566901408451


In [0]:
def test(loaders, model, criterion, use_cuda):

    # Monitor test loss and accuracy
    test_loss = 0.
    correct = 0.
    total = 0.
    
    TP = 0
    FP = 0
    TN = 0
    FN = 0

    model.eval()
    for batch_idx, (data, target) in tqdm(enumerate(loaders['test'])):
        # Move to GPU
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        
        # Find the model's prediction
        output = model(data)
        
        # Calculate the loss
        loss = criterion(output, target)
        
        # Update average test loss 
        test_loss = test_loss + ((1 / (batch_idx + 1)) * (loss.data - test_loss))
        
        # Update the test accuracy
        pred = output.data.max(1, keepdim=True)[1]
        
        for idx in range(len(target)):
          if target[idx] == 1:
            if pred[idx] == 1:
              TP += 1
            else:
              FN += 1
          else:
            if pred[idx] == 0:
              TN += 1
            else:
              FP += 1
    accuracy = (TP+TN)/(TP+FP+FN+TN)
    precision = TP/(TP+FP)
    recall = TP/(TP+FN)
    f1 = 2*(recall * precision) / (recall + precision)
     
    # Print test loss and accuracy
    print('accuracy: {}, precision: {}, recall: {}, f1: {}'.format(accuracy,precision,recall,f1))